In [1]:
import fastai
from fastai import *
from fastai.text import * 
from fastai.text.models import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score

import nltk
from nltk.corpus import stopwords
import re 
from bs4 import BeautifulSoup

from nltk.stem.snowball import SnowballStemmer
from pandarallel import pandarallel

In [2]:
l = load_learner('.','export.pkl')

In [3]:
olddata = pd.read_pickle('newdata.pkl')

In [53]:
newdata_eng = olddata[olddata['Language']!= 'eng']

In [54]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [55]:
#PREPROCEESING THE TEXT

def text_preprocessing_simple(text):
    # remove non-letters
    text_non_letters = re.sub("[^a-zA-Z']", " ", text) 
    # convert to lower-case
    text_lower = text_non_letters.lower()
    return text_lower

In [56]:
def get_preds_confidence(text):
    preds = l.predict(text)
    pred_class = str(preds[0])
    pred_max = preds[2].max().item()
    return (pred_class, pred_max)

In [57]:
newdata_eng['Text'] = newdata_eng['Text'].apply(lambda x: text_preprocessing_simple(x))

/Users/arul/anaconda3/envs/ath/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
from tqdm.notebook import trange, tqdm
tqdm.pandas()

/Users/arul/anaconda3/envs/ath/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
newdata_eng['pred']= newdata_eng['Text'].parallel_apply(get_preds_confidence)

In [ ]:
newdata_eng['confidence'] = newdata_eng['pred'].apply(lambda x: x[1])

In [ ]:
newdata_eng['pred'] = newdata_eng['pred'].apply(lambda x: x[0])